In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import numpy as np
!pip install --upgrade mplfinance
import mplfinance as mpf
%cd /content/TA
from technical_indicator import TA

In [ ]:

def data_pre_inf(nick, dir = "/content/drive/MyDrive/data/naphtha/Crude Oil WTI Futures Historical Data.csv", vol = True):
    data = pd.read_csv(dir)
    if vol:
        if (data.dtypes['Vol.'] == 'O'):
            data['Vol.'] = data['Vol.'].str.rstrip('K')
            data['Vol.'] = data['Vol.'].str.rstrip('M')
            data['Vol.'] = data['Vol.'].replace({'-': 0})

            data['Vol.'] = data['Vol.'].astype('float64')#, copy=False)

    data['Change %'] = data['Change %'].str.rstrip('%')
    data['Change %'] = data['Change %'].astype('float64')#, copy=False)
    if data.dtypes['Price'] == 'O':
        data['Price'] = data['Price'].map(lambda x: re.sub(r',', '', x))
        data['Price'] = data['Price'].astype('float64')#, copy=False)

    data['date'] = pd.to_datetime(data['Date']).astype('datetime64[ns]')
    data = data.drop(['Date'], axis= 1)
    data = data.sort_values(['date'])
    data = data.iloc[:-1,:]
    data = data.replace({'-': 0})
    data = data.drop(["Open", "Low", "High", "Change %"], axis = 1)
    if vol:
        data.columns = ['Price_' + nick, 'Vol_'+nick, 'date']
    else:
        data.columns = ['Price_' + nick, 'date']

    print(nick, data.shape)
    return data

In [ ]:

# dir_list = ["Crude Oil WTI Futures Historical Data.csv",
#             "EUR_USD Historical Data.csv",
#             "Ethanol Futures Historical Data.csv",
#             "GOOGL Historical Data.csv",
#             "Natural Gas Futures Historical Data.csv",
#             "S&P 500 Historical Data.csv",
#             "US 30 Futures Historical Data.csv",
#             "United States 10-Year Bond Yield Historical Data.csv",
#             "United States 2-Year Bond Yield Historical Data.csv"]

# data_wti = data_pre_inf("wti","/content/drive/MyDrive/data/naphtha/" + dir_list[0])
# data_eur_usd = data_pre_inf("eur_usd", "/content/drive/MyDrive/data/naphtha/" + dir_list[1], vol = False)
# data_ethan = data_pre_inf("ethan","/content/drive/MyDrive/data/naphtha/" + dir_list[2])
# data_googl = data_pre_inf("googl","/content/drive/MyDrive/data/naphtha/" + dir_list[3])
# data_gas = data_pre_inf("gas","/content/drive/MyDrive/data/naphtha/" + dir_list[4])
# data_snp500 = data_pre_inf("snp","/content/drive/MyDrive/data/naphtha/" + dir_list[5])
# data_us30 = data_pre_inf("us30","/content/drive/MyDrive/data/naphtha/" + dir_list[6])
# data_us10bond = data_pre_inf("us10bond","/content/drive/MyDrive/data/naphtha/" + dir_list[7], vol = False)
# data_us2bond = data_pre_inf("us2bond","/content/drive/MyDrive/data/naphtha/" + dir_list[8], vol = False)



In [24]:
# %cd /content/finta/finta
# import finta
from pandas_datareader import data as web
! git clone https://github.com/superspray/TA


Cloning into 'TA'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 4), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [25]:
# Get the data from remote source
data = web.DataReader('^SPX', 'stooq')
# Finta likes lowercase
data.columns = ["open", "high", "low", "close", "volume"]
data.sort_values('Date', inplace = True)
data

,open,high,low,close,volume
Date,,,,,
2016-08-05,2168.79,2182.87,2168.79,2182.87,2035038889
2016-08-08,2183.76,2185.44,2177.85,2180.89,1848638889
2016-08-09,2182.24,2187.66,2178.61,2181.74,1852388889
2016-08-10,2182.81,2183.41,2172.00,2175.49,1808305556
2016-08-11,2177.97,2188.45,2177.97,2185.79,1901755556
...,...,...,...,...,...
2021-07-28,4402.95,4415.47,4387.01,4400.64,2118843918
2021-07-29,4403.59,4429.97,4403.59,4419.15,2019354788
2021-07-30,4395.12,4412.25,4389.65,4395.26,2245846882


In [ ]:
# calc bol band
bbands = TA.BBANDS(data, 30)

# cherry pick what to show on the chart
bands_plot = pd.concat([bbands.BB_UPPER, bbands.BB_LOWER], axis=1)

apd = mpf.make_addplot(bands_plot.tail(300))

mpf.plot(data.tail(300), type='candle', style='charles',
        title='SPX BBANDS(30)',
        ylabel='Price (USD)',
        ylabel_lower='Volume',
        volume=True,
        figscale=1.5,
        addplot=apd
        )


In [87]:
# (2020) https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib

Selecting previously unselected package libta-lib0.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 270 kB 29.7 MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.21-cp37-cp37m-linux_x86_64.whl size=1444979 sha256=6855f5f1db71e13e58da30b0b88cc361568c313285500d8c6d355bc940ca7b7e
  Stored in directory: /root/.cache/pip/wheels/32/3c/86/8dfaee7c11df54449f188172fcf66ae6d134f3118d0237df4a
Successfully built ta-lib


# Ta-Lib 과 비교 

In [ ]:
from talib.abstract import *

# uses close prices (default)
output = MINUS_DM(data, timeperiod=14)
output[14:]

## Wilder Smoothing 

In [91]:

def Wilder(data, periods = 3):
    start = np.where(~np.isnan(data))[0][0] #Check if nans present in beginning
    Wilder = np.array([np.nan]*len(data))
    Wilder[start+periods-1] = data[start:(start+periods)].mean() #Simple Moving Average
    for i in range(start+periods,len(data)):
        Wilder[i] = (Wilder[i-1]*(periods-1) + data[i])/periods #Wilder Smoothing
    
    return Wilder


Wilder(TR1)[:10]*3

array([        nan,         nan,         nan, 35.26      , 34.91666667,
       36.23777778, 31.01851852, 30.43901235, 32.3026749 , 36.1151166 ])

In [ ]:
ohlc = data

def Wilder(data, periods = 14, adjust = False):
    start = np.where(~np.isnan(data))[0][0] #Check if nans present in beginning
    Wilder = np.array([np.nan]*len(data))
    Wilder[start+periods-1] = data[start:(start+periods)].sum() #Simple Moving Average
    
    if adjust:
        for i in range(start+periods,len(data)):
            Wilder[i] = (Wilder[i-1]*(periods-1) + data[i]) #Wilder Smoothing
    else:    
        for i in range(start+periods,len(data)):
            Wilder[i] = Wilder[start+periods-1] -Wilder[i-1]/periods+ data[i] #Wilder Smoothing
        
    return Wilder/periods


def Wilder(data, periods):
    start = np.where(~np.isnan(data))[0][0] #Check if nans present in beginning
    Wilder = np.array([np.nan]*len(data))
    Wilder[start+periods-1] = data[start:(start+periods)].mean() #Simple Moving Average
    for i in range(start+periods,len(data)):
        Wilder[i] = (Wilder[i-1]*(periods-1) + data[i])/periods #Wilder Smoothing
    return Wilder


# up = ohlc["high"].diff()
# dn = -ohlc["low"].diff()



# pos = ((up > dn) & (up > 0)) * up
# neg = ((dn > up) & (dn > 0)) * dn


# ## Smoothed
# TR1 = TechA.TR(ohlc)
# TR14 = Wilder(TR1, 14)

# pos14 = Wilder(pos, 14)
# neg14 = Wilder(neg, 14)

# pos_di = pos14 / TR14 * 100 
# neg_di = neg14 / TR14 * 100 

# dx = 100 * np.abs((pos_di - neg_di)/(pos_di + neg_di))
# dmi = (pos14 - neg14)/(pos14 + neg14) * 100
# adx = Wilder(dx, 14, adjust = True)
# adx